Before running the spot detection notebook, run the first notebook, "Read_files_create_batch.ipynb'.

In this notebook we will detect the fish spots using an automatic method called UFISH.
In the next notebook, we will detect the fish spots using a  semi-automatic method called BIGFISH.


WARNING:  run this notebook in the ufish_env.


23/05/25     Jacques Bourg @ Florian Muller lab. Institut Pasteur.

<img src="./HOX_pipeline.png" alt="HOX pipeline" width="1200" height="477">

In [1]:
import os
import sys
import numpy as np
from pathlib import Path
import skimage.io as io
import napari

import ipywidgets as widgets
from IPython.display import display

In [2]:
%load_ext autoreload
%autoreload 2

base_dir = Path("../../src").resolve()
sys.path.append(str(base_dir))
sys.path.append(str(base_dir / "utils"))
sys.path.append(str(base_dir / "detection_fish"))
sys.path.append(str(base_dir / "segmentation"))

from utils.parameters_tracking import Parameter_tracking as Track
from utils.file_handling import FileProcessor
from detection_fish.detect_fish_spots_ufish_env import det_spots_ufish

tk  = Track()
fp  = FileProcessor()

In [3]:
var = str(Path('../Analysis'))
batch_folders = os.listdir(var)
dropdown = widgets.Dropdown(options=batch_folders, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown)

Dropdown(description='Select:', layout=Layout(min_width='150px', width='auto'), options=('.gitkeep', 'test'), …

In [4]:
n         = np.where(np.array(batch_folders) == dropdown.value)[0][0]
file_path = str(Path(var) / Path(batch_folders[n]) / Path(batch_folders[n] +'.json'))
constants = tk.load_json(file_path)
batch_name= constants['BATCH_NAME']; print(batch_name)

test


In [5]:
modalities = constants['MODALITIES']
dropdown2 = widgets.Dropdown(options=modalities, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown2)

Dropdown(description='Select:', layout=Layout(min_width='150px', width='auto'), options=('EXPERIMENT', 'CONTRO…

In [6]:
n2       = np.where(np.array(modalities) == dropdown2.value)[0][0]
modality = modalities[n2]; print(modality)

EXPERIMENT


In [7]:
channel_cell = constants['CHANNELS'] # (Doesn't make sense to select DAPI)
dropdown3    = widgets.Dropdown(options=channel_cell, description='Select:', layout=widgets.Layout(width='auto', min_width='150px')); display(dropdown3)

Dropdown(description='Select:', layout=Layout(min_width='150px', width='auto'), options=('DAPI', 'FISH0', 'FIS…

In [8]:
n3     = np.where(np.array(channel_cell) == dropdown3.value)[0][0]
chan_c = channel_cell[n3]; print(chan_c)

FISH0


In [9]:
batch_fish = constants[f'BATCH_{modality}_{chan_c}']; print(batch_fish);

['../Acquisition/test/EXPERIMENT/FISH0/HM40-J3DIFF-NT6h-HOXA10-620_HOXC8-520-1002-1_FISH0.tif', '../Acquisition/test/EXPERIMENT/FISH0/HM40-J3DIFF-NT24h-HOXA10-620_HOXC8-520-1005-1_FISH0.tif', '../Acquisition/test/EXPERIMENT/FISH0/HM40-J3DIFF-NT48h-HOXA10-620_HOXC8-520-1008-1_FISH0.tif', '../Acquisition/test/EXPERIMENT/FISH0/HM40-J3DIFF-GDF6h-HOXA10-620_HOXC8-522-1_FISH0.tif', '../Acquisition/test/EXPERIMENT/FISH0/HM40-J3DIFF-GDF24h-HOXA10-620_HOXC8-525-1_FISH0.tif', '../Acquisition/test/EXPERIMENT/FISH0/HM40-J3DIFF-GDF48h-HOXA10-620_HOXC8-528-1_FISH0.tif', '../Acquisition/test/EXPERIMENT/FISH0/HM40-J3DIFF-FGFGDF6h-HOXA10-620_HOXC8-531-1_FISH0.tif', '../Acquisition/test/EXPERIMENT/FISH0/HM40-J3DIFF-FGFGDF24h-HOXA10-620_HOXC8-534-1_FISH0.tif', '../Acquisition/test/EXPERIMENT/FISH0/HM40-J3DIFF-FGFGDF48h-HOXA10-620_HOXC8-537-1_FISH0.tif']


In [10]:
spots_folder = Path(f'../Analysis/{batch_name}/{modality}/{chan_c}/spots')
if not spots_folder.exists():
    spots_folder.mkdir(parents=True)
print(spots_folder)

spots_ufish_folder = Path(f'../Analysis/{batch_name}/{modality}/{chan_c}/spots/UFISH')
if not spots_ufish_folder.exists():
    spots_ufish_folder.mkdir(parents=True)
print(spots_ufish_folder)

../Analysis/test/EXPERIMENT/FISH0/spots
../Analysis/test/EXPERIMENT/FISH0/spots/UFISH


In [11]:
#a) UFISH (without deconvolution)

base_dir        = str(Path().resolve().parent.parent)
dots_ufish_path = spots_ufish_folder / f"dots_{modality}_{chan_c}_UFISH.npy"
viewer4         = napari.Viewer(title="UFISH on raw data")
dict_ufish_dots = {}
dict_ufish_dots_local_copy = {}  # this dict contains the same information but the keys are different

counter = 0
for ind_l, file in enumerate(batch_fish):
    file            = Path(file)
    file_abs_path   = str(Path(batch_fish[ind_l]).resolve())
    im_rna          = io.imread(batch_fish[ind_l])
    spots_uf_raw    = det_spots_ufish(file_abs_path)
    
    if im_rna.ndim == 3:
        mip_im_rna = np.max(im_rna, axis=0)    
    elif im_rna.ndim == 2:
        mip_im_rna = im_rna
    
    if np.shape(spots_uf_raw)[1] == 3:
        spots_uf_raw_2d = spots_uf_raw[:,1:]
    elif np.shape(spots_uf_raw)[1] == 2:    
        spots_uf_raw_2d = spots_uf_raw
    
    val = np.percentile(mip_im_rna, 99)    
    viewer4.add_image(mip_im_rna, contrast_limits=(0, val), rgb=False, name=f"{modality} {file.stem}", opacity=0.8)
    viewer4.add_points(spots_uf_raw_2d, name=f"Spots {modality} {file.stem}", size=4,  border_color='#FF0000', face_color=[0, 0, 0, 0]);

    if counter != 0:
        viewer4.layers[f"{modality} {file.stem}"].visible       = False
        viewer4.layers[f"Spots {modality} {file.stem}"].visible = False 

    base_name = '_'.join(file.stem.split('_')[:-1])
    dict_ufish_dots[base_name] = spots_uf_raw
    dict_ufish_dots_local_copy[file.stem] = spots_uf_raw
    counter += 1

format_spots = 'spots_IDzyx'
fp.save_spots_distributed_files(dots_ufish_path, format_spots, dict_ufish_dots, im_rna.ndim)

2025-06-16 08:34:01.622 | INFO     | ufish.api:init_model:70 - Initializing ufish model with kwargs: {}
2025-06-16 08:34:01.623 | INFO     | ufish.api:init_model:72 - Number of parameters: 162959
2025-06-16 08:34:01.807 | INFO     | ufish.api:init_model:78 - CUDA is available, using GPU.
2025-06-16 08:34:01.808 | INFO     | ufish.api:_load_pth_file:232 - Loading weights from /home/jacques/Documents/FISH/Data_analysis/pipeline_smfish_in_apifish/pipeline_HOX_apiFISH/src/detection_fish/v1.0-alldata-ufish_c32.pth
/home/jacques/miniconda3/envs/ufish_env/lib/python3.10/site-packages/ufish/api.py:234: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_

In [14]:
exec(f"DOTS_UFISH_{modality}_{chan_c}_PATH   = dots_ufish_path", globals())
exec(f"SPOTS_FOLDER_{modality}_{chan_c}_PATH = spots_folder")
exec(f"SPOTS_FORMAT = format_spots")

In [15]:
constants2 = tk.collect_constants()
tk.save_constants_and_commit_hash(constants2, batch_name, folder_path = Path(f"../Analysis/{batch_name}"))